In [12]:
import sqlite3, os, csv

try:
  if os.path.exists("db/db.sqlite3"):
    os.remove("db/db.sqlite3")
except:
  conn.close()
  if os.path.exists("db/db.sqlite3"):
    os.remove("db/db.sqlite3")

try:
  conn = sqlite3.connect("db/db.sqlite3")
  #conn = sqlite3.connect(":memory:")

except:
  conn.close()
  conn = sqlite3.connect("db/db.sqlite3")

In [13]:
## CREATE TABLES

# SMARTFAB PAYROLL DUMMY DB

# payroll_user
conn.execute('''CREATE TABLE payroll_user
            (payroll_id     INT PRIMARY KEY NOT NULL,
            first_name      TEXT            NOT NULL,
            last_name       TEXT            NOT NULL,
            employment_status   BOOLEAN     NOT NULL    CHECK (employment_status IN (0, 1)))
''')

# timesheet
conn.execute('''CREATE TABLE timesheet
            (timesheet_id   TEXT    PRIMARY KEY NOT NULL,
            payroll_id      INT                 NOT NULL,
            date            TEXT                NOT NULL,
            shift_start     TEXT                NOT NULL,
            shift_end       TEXT                NOT NULL,
            FOREIGN KEY(payroll_id) REFERENCES payroll_user(payroll_id))
''')

# DASHBOARD RDBMS DATABASE 

# timesheet_entry
conn.execute('''CREATE TABLE timesheet_entry
            (tid                INT     PRIMARY KEY NOT NULL,
            org_timesheet_id    TEXT                NOT NULL,
            payroll_id          INT                 NOT NULL,
            date                TEXT                NOT NULL,
            shift_start         TIME                NOT NULL,
            shift_end           TIME                NOT NULL,
            shift_length        INT                 NOT NULL,
            FOREIGN KEY(org_timesheet_id) REFERENCES timesheet(timesheet_id))
''')
# user_list
conn.execute('''CREATE TABLE user_list
            (payroll_id     INT PRIMARY KEY NOT NULL,
            first_name      TEXT            NOT NULL,
            last_name       TEXT            NOT NULL,
            employment_status   BOOLEAN     NOT NULL    CHECK (employment_status IN (0, 1)))
''')

# user
conn.execute('''CREATE TABLE user
            (uid            INT PRIMARY KEY NOT NULL,
            payroll_id      INT             NOT NULL,
            first_name      TEXT            NOT NULL,
            last_name       TEXT            NOT NULL,
            status          BOOLEAN         NOT NULL    CHECK (status IN (0, 1)),
            timesheet_status    BOOLEAN     NOT NULL    CHECK (timesheet_status IN (0, 1)),
            FOREIGN KEY(payroll_id) REFERENCES payroll_user(payroll_id))
''')

# user_roles
conn.execute('''CREATE TABLE user_roles
            (uid            INT             NOT NULL,
            type_id         INT             NOT NULL)
''')

# user_type
conn.execute('''CREATE TABLE user_type
            (type_id        INT PRIMARY KEY NOT NULL,
            desc            TEXT            NOT NULL,
            level           INT             NOT NULL,
            status          BOOLEAN         NOT NULL    CHECK (status IN (0, 1)))
''')

# machine
conn.execute('''CREATE TABLE machine
            (machine_id     INT PRIMARY KEY NOT NULL,
            desc            TEXT            NOT NULL,
            location_id     INT)''')

# location
conn.execute('''CREATE TABLE location
            (location_id    INT PRIMARY KEY NOT NULL,
            desc            TEXT            NOT NULL)''')

# job
conn.execute('''CREATE TABLE job
            (job_id         INT PRIMARY KEY NOT NULL,
            category        INT             NOT NULL,
            start_date      TEXT            NOT NULL,
            end_date        TEXT)
''')

# category
conn.execute('''CREATE TABLE category
            (category_id    INT PRIMARY KEY NOT NULL,
            arctime         INT             NOT NULL)
''')

# task_type
conn.execute('''CREATE TABLE task_type
            (task_type_id   INT PRIMARY KEY NOT NULL,
            desc            TEXT            NOT NULL)
''')

In [14]:
def ingest(db_file, file, table, num_columns, db_columns):
    try:
        try:
            conn = sqlite3.connect(db_file)

        except:
            conn.close()
            conn = sqlite3.connect(db_file)

        cur = conn.cursor()

        column_count = num_columns - 1
        with open(file, 'r') as f:
            dr = csv.DictReader(f)
            output = []
            for i in dr:
                output.append(tuple([(v) for v in i.values()]))
        insert_string = "INSERT INTO "+ table +" ("+ db_columns +") VALUES (?" +', ?'*column_count +");"
        cur.executemany(insert_string, output)
        conn.commit()

        return "Table "+ table +" ingested from "+ file + " successfully with "+ str(conn.total_changes) +" rows added."

    except:
         raise Exception()

# ingest(db_connection, db_cursor, csv_file, db_table, num_of_columns, column_names)
print(ingest("db/db.sqlite3", 'payroll_users.csv', 'payroll_user', 4, 'payroll_id, first_name, last_name, employment_status'))
print(ingest("db/db.sqlite3", 'timesheet_data.csv', 'timesheet', 5, 'timesheet_id, payroll_id, date, shift_start, shift_end'))
print(ingest("db/db.sqlite3", 'timesheets.csv', 'timesheet_entry', 7, 'tid, org_timesheet_id, payroll_id, date, shift_start, shift_end, shift_length'))
print(ingest("db/db.sqlite3", 'user_list.csv', 'user_list', 4, 'payroll_id, first_name, last_name, employment_status'))
print(ingest("db/db.sqlite3", 'user_types.csv', 'user_type', 4, 'type_id, desc, level, status'))
print(ingest("db/db.sqlite3", 'machines.csv', 'machine', 3, 'machine_id, desc, location_id'))
print(ingest("db/db.sqlite3", 'locations.csv', 'location', 2, 'location_id, desc'))
print(ingest("db/db.sqlite3", 'categories.csv', 'category', 2, 'category_id, arctime'))
print(ingest("db/db.sqlite3", 'jobs.csv', 'job', 4, 'job_id, category, start_date, end_date'))
print(ingest("db/db.sqlite3", 'task_types.csv', 'task_type', 2, 'task_type_id, desc'))


Table payroll_user ingested from payroll_users.csv successfully with 99 rows added.
Table timesheet ingested from timesheet_data.csv successfully with 1499 rows added.
Table timesheet_entry ingested from timesheets.csv successfully with 1499 rows added.
Table user_list ingested from user_list.csv successfully with 99 rows added.
Table user_type ingested from user_types.csv successfully with 4 rows added.
Table machine ingested from machines.csv successfully with 25 rows added.
Table location ingested from locations.csv successfully with 6 rows added.
Table category ingested from categories.csv successfully with 5 rows added.
Table job ingested from jobs.csv successfully with 250 rows added.
Table task_type ingested from task_types.csv successfully with 3 rows added.


In [15]:
conn.close()